# Comparison of Effect between Treatment A and B

In [ ]:
# Enable the commands below when running this program on Google Colab.
# !pip install arviz==0.7
# !pip install pymc3==3.8
# !pip install Theano==1.0.4
 
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import pymc3 as pm

import math

plt.style.use('seaborn-darkgrid')
np.set_printoptions(precision=3)
pd.set_option('display.precision', 3)

In [ ]:
data = pd.DataFrame([[90, 25], [78, 39]], columns=['Effective', 'Not effective'], index=['Treatment A', 'Treatment B'])
display(data)

## Bayesian analysis

In [ ]:
with pm.Model() as model:
    # Prior distribution
    p1 = pm.Uniform('p1', 0, 1)
    p2 = pm.Uniform('p2', 0, 1)

    # Likelihood
    y1_pred = pm.Binomial('y1_pred', n=data.loc['Treatment A'].sum(), p=p1, observed=data.loc['Treatment A']['Effective'])
    y2_pred = pm.Binomial('y2_pred', n=data.loc['Treatment B'].sum(), p=p1, observed=data.loc['Treatment B']['Effective'])

    # Difference of proportions
    diff = pm.Deterministic('diff', p1 - p2)

    # Ratio of proportions
    ratio = pm.Deterministic('ratio', p1 / p2)

    # Odds
    odds1 = pm.Deterministic('odds1', p1 / (1 - p1))
    odds2 = pm.Deterministic('odds2', p2 / (1 - p2))

    # Odds ratio
    odds_ratio = pm.Deterministic('odds_Ratio', odds1 / odds2)

    trace = pm.sample(21000, chains=5)

In [ ]:
chain = trace[1000:]
pm.traceplot(chain)
plt.show()

In [ ]:
pm.summary(chain)

In [ ]:
print('Effective rate (Treatment A): {:.3f} % ({:.3f})'.format(chain['p1'].mean() * 100, chain['p1'].std()))

print('Effective rate (Treatment B): {:.3f} % ({:.3f})'.format(chain['p2'].mean() * 100, chain['p2'].std()))

print('Difference: {:.3f} % ({:.3f})'.format(chain['diff'].mean() * 100, chain['diff'].std()))


In [ ]:
print('Probability of effective rate of A is 0.1 higher than that of B: {:.3f} %'.format((chain['diff'] > 0.1).mean() * 100))
# 比率の差が0.1より大きい確率はそれほど高くないので、「治療法Aの方が効果がある」と断言するのは控えた方が良い